# Klasyfikacja znaków drogowych z wykorzystaniem DCNN

Uwaga: ćwiczenie powstało w oparciu o następujący [tutorial](https://www.pyimagesearch.com/2019/11/04/traffic-sign-classification-with-keras-and-deep-learning/).
Osoby zainteresowane tematem - moim zdaniem (Tomasz Kryjak) powinien to być każdy z Państwa - zachęcam do prześledzenia tego dokumentu bardziej szczegółowo.

Inne podobne tutoriale:
- https://chsasank.github.io/keras-tutorial.html
- https://towardsdatascience.com/building-a-road-sign-classifier-in-keras-764df99fdd6a


To krótkie ćwiczenie nie zastąpi pełnego kursu z AI, ale powinno pozwolić poznać podstawowe etapy związane z inżynierskim wykorzystaniem głębokich konwolucyjnych sieci neuronowych (**DCNN** - Deep Convolutional Neural Network).

## Założenia

Poniższy notebook jest kompletny i gotowy do uruchomienia.
Jedyne wyzwania (niekoniecznie trywialne) to sprawy techniczne.
W przyszłości będzie to ulegało zmianom.

Uwaga - czas obliczeń może być znaczny, ale można go wykorzystać np. na czytanie wskazanego tutoriala, tudzież analizę kodu.

## Instalacja i sprawy techniczne

Notebook do działania potrzebuje pakietów:
- OpenCV,
- NumPy,
- scikit-learn,
- scikit-image,
- imutils,
- matplotlib,
- TensorFlow 2.0 (CPU lub GPU).

Opcje uruchomienia są dwie:
- _Google Colaboratory_ - tam wszystko jest zainstalowane + mamy zasoby obliczeniowe, ale trzeba nieco pokombinować z dostarczeniem danych. Opis tej metody jest przedstawiony poniżej. Jest to opcja rekomendowana.
- lokalnie (instalacja pakietów via *pip* lub poprzez PyCharm).

Dodatkowo należy pobrać bazę danych [GTSRB](https://drive.google.com/file/d/1EQ-tyVHIdVaa4_1bob1zv8zgaVmvyyqV/view?usp=sharing) (German Traffic Sign Recognition Benchmark) - waży ona 300 MB.
Baza zawiera ponad 50000 obrazków dla 43 klas znaków.
Ma też niestety dwie istotne wady:
- różna liczba przykładów z poszczególnych klas (od 180 do ponad 2000),
- część znaków stanowi duże wyzwanie (słaba jakość, kontrast) - uczciwie mówiąc, niektóre trudno rozpoznać.

Nie wchodząc zbytnio w szczegóły: sieć DCNN "uczy się" (jest uczona) na podstawie przykładów, podobnie jak uczymy się i my.
I teraz np. jeśli rozwiążemy 10 zadań dotyczących całek i 100 zadań na temat pochodnych, to co na egzaminie nam wyjdzie lepiej?

Stąd tego typu dysproporcja stanowi problem.
Słaba jakość zdjęć też utrudnia uczenie.

**W przypadku przetwarzania lokalnego bazę należy rozpakować.**


## Informacje wstępne o DCNN

Co to jest sieć DCNN (Deep Convolutional Neural Network)?

Jest to model naśladujący działanie ludzkiego mózgu (tu konkretnie: sposobu przetwarzania informacji wizyjnej).
Przedstawienie całej teorii w notatniku nie jest specjalnie wygodne - zainteresowane osoby odsyłam do obszernej literatury.
Bez wchodzenia w szczegóły, sieć można traktować jako czarną skrzynkę, która na wejściu dostaje obraz, a na wyjściu wyniki (klasy obiektów).

Należy pamiętać, że sieć trzeba _nauczyć_.
W uproszczeniu proces uczenia polega na tym, że prezentujemy sieci obraz, otrzymujemy jakiś wynik, porównujemy go z pożądanym i według specjalnego algorytmu modyfikujemy parametry sieci (tzw. _wsteczna propagacja błędu_).
Uwaga: opisano tzw. uczenie nadzorowane (z "nauczycielem").
Istnieje też uczenie bez nauczyciela (sieci samouczące) oraz ze wspomaganiem (reinforcement learning).
Szczególnie to drugie jest bardzo ciekawe - warto sobie o tym poczytać.
To właśnie ta metodologia stoi za sukcesami **AlphaGo** (w grze Go) czy też **AlphaStar** (w grze Starcraft).

Mając nauczoną sieć, można przeprowadzić tzw. wnioskowanie (ang. *inference*).

Warto też wiedzieć, że sieci konwolucyjne to jest jedna z możliwości, dedykowana dla obrazów.
Dla innych zagadnień stosowane są inne modele.
Ponadto w ramach samych DCNN występuje wiele różnych rozwiązań, choć są one zbudowane z mniej więcej podobnych "klocków".

Źródła dodatkowych informacji:
- https://en.wikipedia.org/wiki/Convolutional_neural_network,
- https://d2l.ai/,
- kursy na _Coursera_,
- oraz naprawdę bardzo dużo innych źródeł.


## Co dzisiaj zrobimy?

Przejdziemy przez następujące kroki:
- utworzenie modelu (definicja architektury sieci),
- implementacja funkcji do przygotowania zbioru uczącego,
- przygotowanie danych,
- uczenie,
- analiza wyników uczenia,
- testy - wnioskowanie.

Bardziej szczegółowe komentarze są umieszczone w tekście.


In [1]:
# potrzebne biblioteki
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import classification_report
from skimage import transform
from skimage import exposure
from skimage import io

from imutils import paths

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import os
import pickle
import requests
import imutils
import cv2

matplotlib.use("Agg")

# Definicja architektury sieci do klasyfikacji
# Sieć składa się z:
# 5 warstw konwolucyjnych - Conv2D, po nich funkcja aktywacji (relu), normalizacja oraz podpróbkowanie (zmiana rozdzielczości)
# 2 warstw tzw. w pełni połączonych

# Uwaga: sieć ta jest zbliżona do rozwiązania AlexNet (https://en.wikipedia.org/wiki/AlexNet).
# Sam AlexNet natomiast to jedna z pierwszych (i na pewno najbardziej znanych) sieci konwolucyjnych.
# Jest tak m.in. przez sukces tego rozwiązania w konkursie ImageNet Large Scale Visual Recognition Challenge (ISLVRC).
# Artykuł opisujący tę sieć ma ponad 70000 cytowań.

class TrafficSignNet:
		@staticmethod
		def build(width, height, depth, classes):
				model = Sequential()
				inputShape = (height, width, depth)
				chanDim = -1

				# CONV => RELU => BN => POOL
				model.add(Conv2D(8, (5, 5), padding="same", input_shape=inputShape))
				model.add(Activation("relu"))
				model.add(BatchNormalization(axis=chanDim))
				model.add(MaxPooling2D(pool_size=(2, 2)))

				# first set of (CONV => RELU => BN) * 2 => POOL
				model.add(Conv2D(16, (3, 3), padding="same"))
				model.add(Activation("relu"))
				model.add(BatchNormalization(axis=chanDim))
				model.add(Conv2D(16, (3, 3), padding="same"))
				model.add(Activation("relu"))
				model.add(BatchNormalization(axis=chanDim))
				model.add(MaxPooling2D(pool_size=(2, 2)))

				# second set of (CONV => RELU => BN) * 2 => POOL
				model.add(Conv2D(32, (3, 3), padding="same"))
				model.add(Activation("relu"))
				model.add(BatchNormalization(axis=chanDim))
				model.add(Conv2D(32, (3, 3), padding="same"))
				model.add(Activation("relu"))
				model.add(BatchNormalization(axis=chanDim))
				model.add(MaxPooling2D(pool_size=(2, 2)))

				# freshly added set of (CONV => RELU => BN) * 2 => POOL
				model.add(Conv2D(64, (3, 3), padding="same"))
				model.add(Activation("relu"))
				model.add(BatchNormalization(axis=chanDim))
				model.add(Conv2D(64, (3, 3), padding="same"))
				model.add(Activation("relu"))
				model.add(BatchNormalization(axis=chanDim))
				model.add(MaxPooling2D(pool_size=(2, 2)))

				# first set of FC => RELU => BN layers
				model.add(Flatten())
				model.add(Dense(128))
				model.add(Activation("relu"))
				model.add(BatchNormalization())
				model.add(Dropout(0.5))

				# second set of FC => RELU => BN layers
				model.add(Flatten())
				model.add(Dense(128))
				model.add(Activation("relu"))
				model.add(BatchNormalization())
				model.add(Dropout(0.5))

				# softmax classifier
				model.add(Dense(classes))
				model.add(Activation("softmax"))

				return model

print("[INFO] Model created!")

[INFO] Model created!


In [2]:
# funkcja do przygotowania obrazów
def load_split(basePath, csvPath):
    # inicjalizacja list dla danych i etykiet (klas znaków)
    data = []
    labels = []

	  # wczytanie zawartości pliku CSV z opisem danych, z pominięciem pierwszej linii
    rows = open(csvPath).read().strip().split("\n")[1:]

    # wymieszanie przykładów uczących
    random.shuffle(rows)

    # pętla po przykładach uczących
    for (i, row) in enumerate(rows):
	      # wypisanie informacji co 1000 przykładów
        if i > 0 and i % 1000 == 0:
           print("[INFO] processed {} total images.".format(i))

        # dla danego rzędu pozyskujemy etykietę (label) oraz ścieżkę do pliku
        (label, imagePath) = row.strip().split(",")[-2:]

	  	  # "skompletowanie" ścieżki i wczytanie obrazu
        imagePath = os.path.sep.join([basePath, imagePath])
        image = io.imread(imagePath)

        # przeskalowanie do rozmiaru 32x32 i poprawa kontrastu metodą CLAHE (Contrast Limited Adaptive Histogram Equalization)
        image = transform.resize(image, (32, 32))
        image = exposure.equalize_adapthist(image, clip_limit=0.1)

        # dodanie obrazka i etykiet do listy
        data.append(image)
        labels.append(int(label))

    # konwersja danych i etykiet na tablice NumPy
    data = np.array(data)
    labels = np.array(labels)

	  # zwracamy dane i etykiety (w formie krotki)
    return (data, labels)

print("[INFO] Function defined.")

[INFO] Function defined.


In [3]:
!pip3 install --upgrade gdown

In [4]:
# pobranie pliku z nazwami znaków
url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/14_TSR_DCNN/'
fileNames = ["signnames.csv"]

for fileName in fileNames:
    if not os.path.exists(fileName):
        r = requests.get(url + fileName, allow_redirects=True)
        open(fileName, 'wb').write(r.content)

# Przygotowanie danych
# Jeśli ktoś używa Google Colab, należy:
# 1. Wgrać plik gtsrb.zip na dysk Google'a.
# 2. Podmontować dysk.

# from google.colab import drive

# drive.mount('/content/gdrive')

!gdown 1b-ijBmRyyyL6ypu9Silp5RZz90sFc4VD&confirm=t

# 3. Zainstalować zip i rozpakować plik (uwaga, trzeba ustawić ścieżkę)
!apt install unzip
!unzip 'gtsrb.zip'

# Jeśli ktoś pracuje lokalnie, to trzeba tu ustawić ścieżkę do danych.
dataset = "gtsrb/"

# wczytanie nazw etykiet
labelNames = open("signnames.csv").read().strip().split("\n")[1:]
labelNames = [l.split(",")[0] for l in labelNames]

# ustawienie ścieżki do zbioru uczącego i testowego
trainPath = os.path.sep.join([dataset, "Train.csv"])
testPath = os.path.sep.join([dataset, "Test.csv"])

# wczytanie danych uczących i testowych (może dość długo trwać)
print("[INFO] loading training and testing data...")
(trainX, trainY) = load_split(dataset, trainPath)
(testX, testY) = load_split(dataset, testPath)

# przeskalowanie danych do zakresu [0; 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# Zakodowanie etykiet danych uczących i testowych w formacie one-hot-encoding (z całego wektora tylko jedna wartość to 1, reszta 0).
# To wprost koresponduje z wyjściem z sieci (warstwa softmax), gdzie otrzymujemy wektor długości takiej, ile mamy klas (tutaj: 43)
# i wyszukujemy w nim maksimum.

numLabels = len(np.unique(trainY))
trainY = to_categorical(trainY, numLabels)
testY = to_categorical(testY, numLabels)

# zapis danych uczących i testowych do pliku (żeby tego ewentualnie nie powtarzać), jak coś na dalszym etapie pójdzie nie tak
print("[INFO] saving training and testing data...")

with open('train_test.pickle', 'wb') as f:
    pickle.dump([trainX, trainY, testX, testY], f)

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
  inflating: gtsrb/train/27/00027_00003_00013.png  
  inflating: gtsrb/train/27/00027_00003_00014.png  
  inflating: gtsrb/train/27/00027_00003_00015.png  
  inflating: gtsrb/train/27/00027_00003_00016.png  
  inflating: gtsrb/train/27/00027_00003_00017.png  
  inflating: gtsrb/train/27/00027_00003_00018.png  
  inflating: gtsrb/train/27/00027_00003_00019.png  
  inflating: gtsrb/train/27/00027_00003_00020.png  
  inflating: gtsrb/train/27/00027_00003_00021.png  
  inflating: gtsrb/train/27/00027_00003_00022.png  
  inflating: gtsrb/train/27/00027_00003_00023.png  
  inflating: gtsrb/train/27/00027_00003_00024.png  
  inflating: gtsrb/train/27/00027_00003_00025.png  
  inflating: gtsrb/train/27/00027_00003_00026.png  
  inflating: gtsrb/train/27/00027_00003_00027.png  
  inflating: gtsrb/train/27/00027_00003_00028.png  
  inflating: gtsrb/train/27/00027_00003_00029.png  
  inflating: gtsrb/train/27/00027_00004_00000.png  

In [5]:
# przygotowanie do uczenia modelu
# liczba etykiet
numLabels = trainY.shape[1]

# liczba epok (iteracji algorytmu uczenia)
NUM_EPOCHS = 15

# współczynnik uczenia
INIT_LR = 1e-3

# rozmiar "wsadu" do batch normalization (https://en.wikipedia.org/wiki/Batch_normalization)
BS = 64

# wczytanie zbioru uczącego i testowego
with open('train_test.pickle', 'rb') as f:
    trainX, trainY, testX, testY = pickle.load(f)

# Stworzenie obiektu do augmentacji danych
# Co to jest augmentacja? Ogólnie jest to "sztuczne" zwiększenie liczebności zbioru uczącego.
# Jak można się domyślać po nazwach argumentów, tu obejmuje takie operacje jak: obrót, skalowanie, przesunięcie czy zniekształcenie.

aug = ImageDataGenerator(
		rotation_range=10,
		zoom_range=0.15,
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=False,
		vertical_flip=False,
		fill_mode="nearest"
)

# Inicjalizacja optymalizatora oraz kompilacja modułu
# Parametr LR (Learning Rate) mówi o tym, jak bardzo sieć się uczy (jak bardzo możemy zmienić parametry w danym kroku).
# Proszę zwrócić uwagę, że ustawia się również jego zanikanie (decay).
# Upraszczając (znowu): w poszukiwaniu optimum lokalnego w przestrzeni rozwiązań (bo do tego ostatecznie sprowadza się problem uczenia),
# na początku dopuszczamy duże przesunięcia, a z czasem coraz mniejsze.

print("[INFO] compiling model...")

opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / (NUM_EPOCHS * 0.5))
model = TrafficSignNet.build(width=32, height=32, depth=3, classes=numLabels)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# wyliczanie wag dla klas - wskazanie dla modułu, że występuje problem z różną liczebnością zbioru uczącego
classTotals = trainY.sum(axis=0)
classWeight = classTotals.max() / classTotals
classWeightD = { x: classWeight[x] for x in range(0, classWeight.shape[0]) }

print("[INFO] model ready to learn!")

[INFO] compiling model...


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[INFO] model ready to learn!


In [6]:
# Uczenie modelu
# To może chwilę trwać - ograniczyliśmy liczbę epok (iteracji procesu) do 15.
# Ogólnie jest to etap, który dobrze można akcelerować na GPU, przy czym lokalna konfiguracja TensorFlow/Keras do współpracy z GPU nie jest prosta
# (i na pewno trwa znacznie dłużej niż to uczenie).
# W kolejnych epokach wyświetlają się wskaźniki:
# - accuracy - dokładność na zbiorze uczącym (powinna rosnąć)
# - loss - funkcja błędu dla zbioru uczącego (powinna maleć)
# - val_accuracy - dokładność dla zbioru walidacyjnego (powinna rosnąć)
# - val_loss - funkcja błędu na zbiorze walidacyjnym (powinna maleć)

# Zbiory uczące i testowe są rozłączne, aby móc zaobserwować zjawisko "przeuczenia" modelu (ang. overfitting).
# Najkrócej ujmując, jest to analogia nauki "na pamięć". Model dobrze nauczy się danych uczących, ale gorzej będzie sobie radził na innych.

print("[INFO] training network...")
H = model.fit(
		aug.flow(trainX, trainY, batch_size=BS),
		validation_data=(testX, testY),
		steps_per_epoch=trainX.shape[0] // BS,
		epochs=NUM_EPOCHS,
		class_weight=classWeightD,
		verbose=1
)

# zapis sieci na dysk - coby nie trzeba było drugi raz uczyć
print("[INFO] serializing network to 'trafficsignnet.keras'...")
model.save("trafficsignnet.keras")
print("[INFO] training network done.")

[INFO] training network...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 54s 59ms/step - accuracy: 0.0726 - loss: 9.6484 - val_accuracy: 0.0434 - val_loss: 3.9060
Epoch 2/15
  1/612 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.2031 - loss: 5.7604

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2031 - loss: 5.7604 - val_accuracy: 0.0481 - val_loss: 3.8610
Epoch 3/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 64s 41ms/step - accuracy: 0.3283 - loss: 5.1082 - val_accuracy: 0.5107 - val_loss: 1.4072
Epoch 4/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5000 - loss: 4.8793 - val_accuracy: 0.5255 - val_loss: 1.3705
Epoch 5/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 25s 41ms/step - accuracy: 0.5120 - loss: 3.4229 - val_accuracy: 0.5959 - val_loss: 1.1850
Epoch 6/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5469 - loss: 2.5659 - val_accuracy: 0.5880 - val_loss: 1.2254
Epoch 7/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 40s 41ms/step - accuracy: 0.6350 - loss: 2.4526 - val_accuracy: 0.6190 - val_loss: 1.1924
Epoch 8/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6562 - loss: 1.5788 - val_accuracy: 0.6165 - val_loss: 1.2057
Epoch 9/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 39s 41ms/step - accuracy: 0.7049 - loss: 1.9176 - val_accuracy: 0.732

## Ewaluacja

Mając dany znak i wynik klasyfikacji, mamy 4 możliwości:
- TP (_True Positive_) - wynik klasyfikacji i stan faktyczny są pozytywne,
- FP (_False Positive_) - klasyfikacja wskazuje na znak X, ale stan faktyczny to nie X,
- FN (_False Negative_) - klasyfikacja wskazuje, że to nie znak X, natomiast stan faktyczny to X,
- TN (_True Negative_) - klasyfikacja wskazuje, że to nie znak X i to istotnie nie jest znak X.

Na tej podstawie można konstruować wskaźniki:
- **precision** = TP / (TP + FP)
- **recall** = TP / (TP + FN)
- **f1-score** = 2 * precision * recall / (precision + recall)

Szerszy ich opis można znaleźć na [Wikipedii](https://en.wikipedia.org/wiki/Sensitivity_and_specificity).

Parametr *support* oznacza liczbę próbek.

Warto na chwilę się zastanowić nad tym, co oznaczają te wskaźniki.
Precyzja (precision) będzie tym większa, im mniej będzie FP, czyli sytuacji, że znak różny od X będzie uznany za X (należący do ewaluowanej klasy).
Natomiast czułość (recall) będzie tym większa, im mniej będzie FN, czyli sytuacji, że znak X (należący do rozpatrywanej klasy) nie będzie uznany za X.

Należy zwrócić uwagę, że:
- w idealnym przypadku (brak błędów) oba powinny mieć wartość 1,
- są poniekąd przeciwstawne - wszystko zależy od tego, czy nasz klasyfikator jest mniej, czy bardziej restrykcyjny,
- f1-score, jako średnia harmoniczna, łączy oba wskaźniki.


Proszę jeszcze zwrócić uwagę na rysunek `train.png` - wyświetlić i zastanowić się, co oznacza.


In [7]:
# ewaluacja modelu (sprawdzenie, jak się nauczył)
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))

# wykres funkcji kosztu i dokładności
N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
plt.figure()

plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")

plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch no")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("train.png")

[INFO] evaluating network...
198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
                               precision    recall  f1-score   support

         Speed limit (20km/h)       0.96      0.37      0.53        60
         Speed limit (30km/h)       0.98      0.64      0.77       720
         Speed limit (50km/h)       0.99      0.40      0.57       750
         Speed limit (60km/h)       0.97      0.66      0.78       450
         Speed limit (70km/h)       0.73      0.83      0.78       660
         Speed limit (80km/h)       0.37      0.29      0.33       630
  End of speed limit (80km/h)       0.99      0.63      0.77       150
        Speed limit (100km/h)       0.37      0.81      0.51       450
        Speed limit (120km/h)       0.51      0.91      0.65       450
                   No passing       0.99      0.69      0.81       480
 No passing veh over 3.5 tons       0.89      0.90      0.89       660
 Right-of-way at intersection       0.99      0.49      0.66       420
      

## Wnioskowanie

Kiedy już mamy nauczony model, to możemy go użyć do wnioskowania (ang. inference).
Wtedy na wejście podajemy zdjęcie znaku, a na wyjściu uzyskujemy informację, co to za znak.

Uwaga: proszę utworzyć folder `examples`.

Na samym końcu patrzymy, co nam wyszło.
Z uwagi na ograniczoną liczbę iteracji - "szału nie ma", ale i tak znaki o lepszej jakości powinny być rozpoznane poprawnie.


In [8]:
# wczytujemy model
print("[INFO] loading model...")
model = load_model("trafficsignnet.keras")

# wczytujemy nazwy klas (ponownie)
labelNames = open("signnames.csv").read().strip().split("\n")[1:]
labelNames = [l.split(",")[0] for l in labelNames]

# wczytujemy obrazy, mieszamy je i wybieramy podzbiór
print("[INFO] predicting...")
imagePaths = list(paths.list_images(f"{dataset}/Test"))
random.shuffle(imagePaths)
imagePaths = imagePaths[:25]

# pętla po obrazach
for (i, imagePath) in enumerate(imagePaths):
		# wczytujemy obraz, skalujemy i wyrównujemy histogram - dokładnie tak, jak wcześniej
		image = io.imread(imagePath)
		image = transform.resize(image, (32, 32))
		image = exposure.equalize_adapthist(image, clip_limit=0.1)

		# skalujemy do wartości [0; 1]
		image = image.astype("float32") / 255.0
		image = np.expand_dims(image, axis=0)

		# przeprowadzamy wnioskowanie
		preds = model.predict(image)

		# wybieramy najbardziej prawdopodobną odpowiedź
		j = preds.argmax(axis=1)[0]
		label = labelNames[j]

		# wizualizacja i zapis do pliku
		image = cv2.imread(imagePath)
		image = imutils.resize(image, width=128)
		cv2.putText(image, label, (5, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
		p = os.path.sep.join(["examples", "{}.png".format(i)])
		cv2.imwrite(p, image)

print("[INFO] done.")

[INFO] loading model...
[INFO] predicting...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
[INFO] 